# **Algorithms Homework 2**
**BMI 2005: Dr. Costa**

**Helena Belloff**

**3/31/2020**

**Question 1**

Part A

In [20]:
# First we need to copy our code from HW 1 so we can import the xyz file and edit the previous functions

import numpy as np
import re

def xyz_file(filename):
    # Open file
    with open(filename, 'r') as f:
        num_of_atoms = None
        comment = None
        output = []
        for line in f:
            if num_of_atoms == None:
                num_of_atoms = int(line)
            elif comment == None:
                comment = line
            else:
                split_l = re.split('\s|(?<!\d)[,.](?!\d)', line)
                float_filter = re.compile(r'^-?\d+(?:\.\d*)') 
                fl = [a for a in split_l if float_filter.match(a)] 
                while("" in fl) : 
                    fl.remove("")
                output.append(fl)
        output = np.array(output, dtype = 'float')
        return output

In [21]:
filename = '/Users/helenabelloff/Downloads/lj-0200-liquid.xyz'
liquid = xyz_file(filename)
#print("Liquid File: ")
#print(liquid)

In [22]:
def distance(a, b):
    return np.linalg.norm(a - b)

In [23]:
def V_ij(r):
    return 4 *((1/r)**12 - (1/r)**6)

In [24]:
import numpy as np

boundary_length = np.array([6.4365959, 6.4365959, 6.4365959])

def LJ_Potential_new(atoms, boundary_length, rcut):
    total = 0
    for i in range(0, len(atoms)):
        for j in range(i+1,len(atoms)):
            square_distance = 0.0
            for dim in range(0,len(atoms[0])):
                distance_in_dim = atoms[i][dim] - atoms[j][dim]
                distance_in_dim -= boundary_length[dim] * round(distance_in_dim / boundary_length[dim])
                square_distance += distance_in_dim ** 2
            distance = np.sqrt(square_distance)
            if distance < rcut:
                 total += V_ij(distance)
    return total

Part B

In [25]:
LJ_Potential_new(liquid, boundary_length, rcut = 2.0)

-862.1552244209805

**Question 2**

Part A

In [32]:
import numpy as np

# Need to create a function to call because this is getting way too messy
# Taking a chunk of the code from question 1:

dim_number = 3
def distance_dim(atoms, i, j, dim_number):
    square_distance = 0.0
    for dim in range(0,len(atoms[0])):
        distance_in_dim = atoms[i][dim] - atoms[j][dim]
        distance_in_dim -= boundary_length[dim] * round(distance_in_dim / boundary_length[dim])
        square_distance += distance_in_dim ** 2
    distance = np.sqrt(square_distance)
    return distance

cutoff = 2.0
sheath = 0.3

def Verlet_list(atoms, cutoff, sheath):
    cutoff_plus_sheath = cutoff + sheath
    lookup = np.empty(shape=[atoms.shape[0]]).astype(int)
    value = []
    for i in range(atoms.shape[0]):
        pair_counter = 0
        for j in range(i+1,atoms.shape[0]):
            distance = distance_dim(atoms, i, j, dim_number)
            if distance < cutoff_plus_sheath:
                pair_counter += 1
                value.append(j)
            lookup[i] = pair_counter
    return value, lookup

In [33]:
print(Verlet_list(liquid, cutoff = 2.0, sheath = 0.3))

([4, 10, 14, 19, 36, 42, 43, 46, 51, 56, 62, 65, 70, 71, 72, 73, 83, 84, 87, 89, 92, 93, 102, 105, 110, 113, 117, 123, 136, 146, 149, 150, 157, 169, 176, 177, 184, 185, 186, 192, 194, 197, 6, 12, 16, 40, 47, 54, 56, 69, 80, 85, 86, 87, 92, 94, 106, 107, 112, 116, 121, 122, 125, 127, 137, 139, 150, 152, 153, 154, 155, 159, 160, 166, 168, 172, 173, 181, 183, 185, 194, 3, 11, 17, 23, 24, 25, 30, 33, 37, 38, 47, 53, 54, 57, 60, 61, 67, 80, 81, 82, 89, 90, 96, 98, 101, 104, 108, 109, 124, 154, 160, 161, 162, 163, 164, 168, 174, 177, 180, 183, 186, 187, 195, 12, 17, 23, 24, 27, 33, 37, 38, 45, 50, 60, 61, 67, 73, 78, 79, 81, 85, 88, 89, 96, 97, 98, 102, 108, 109, 110, 118, 143, 145, 146, 148, 149, 161, 162, 164, 167, 177, 178, 179, 180, 182, 186, 190, 193, 10, 16, 19, 20, 32, 34, 35, 43, 46, 51, 56, 65, 70, 72, 73, 77, 83, 84, 87, 92, 93, 100, 105, 110, 113, 115, 126, 136, 140, 144, 150, 157, 169, 176, 185, 191, 192, 194, 197, 7, 8, 10, 22, 34, 35, 39, 40, 44, 63, 66, 74, 76, 84, 111, 112, 1

Part B

In [34]:
def Verlet_Potential(atoms, cutoff, sheath, value, lookup):
    total = 0
    potential = 0.0
    for i in range(0, len(lookup)):
        if i < (len(lookup)-1):
            total_lookup = total+lookup[i]
        else:
            len(value)
        pot_atoms = value[total:total_lookup]
        for j in range(0, len(pot_atoms)):
            pot_atoms_of = pot_atoms[j]
            square_distance = 0.0
            for dim in range(0,len(atoms[0])):
                distance_in_dim = atoms[i][dim] - atoms[pot_atoms_of][dim]
                distance_in_dim -= boundary_length[dim] * round(distance_in_dim / boundary_length[dim])
                square_distance += distance_in_dim ** 2
            distance = np.sqrt(square_distance)
            if distance < cutoff:
                potential += V_ij(distance)
        total += lookup[i]
    return potential

value, lookup = Verlet_list(liquid, 2.0, 0.3)
print(Verlet_Potential(liquid, 2.0, 0.3, value, lookup))

-862.1552244209805


The complexity of this function is O(n) because even if we have a large number of atoms, the verlet list function ensures that the atoms with interactions have calculated distances. So we won't reach a point where it is so many atoms it won't be linear.

**Question 3**

Part A

A cell-based domain decomposition of this box should be larger than the cutoff size, which is 2.0 in this case. We can have a box with a cell decomposition of 3 x 3 x 3.

Part B

In [35]:
class Atom:
    def __init__(atom, atom_idx, pos):
        atom.idx = atom_idx
        atom.pos = pos
        atom.next = None
        
    def getIdx(atom):
        return atom.idx
    
    def getNext(atom):
        return atom.next
    
    def setIdx(atom, atom_idx_new):
        atom.idx = atom_idx_new
        
    def setNext(atom,next_new):
        atom.next = next_new

class LinkedList:
    def __init__(atom):
        atom.head = None
    
    def isEmpty(atom):
        return atom.head == None
    
    def add(atom, atom_idx, pos): 
        front = atom.head
        node.add = Atom(atom_idx, pos)
        atom.head = node.add
        atom.head.setNext(front)
    
    def getIndex(atom, position):
        node = atom.head
        for i in range(1,position):
            node = node.getNext()
        return node

Part C

**Question 4**

In [36]:
import numpy as np

A = np.array([
    [1, 3, 10, 2],
    [14, 13, 12, 5],
    [15, 9, 11, 6],
    [16, 17, 13, 18]
])

lo = 0
hi = A.shape[1] - 1
            
def two_peak(A, lo, hi):
    if len(A) > 1: #cols
        mid = (lo + hi)//2
        max = 0
        for i in range(A.shape[0]): #rows
            # if greater than the max, it becomes new max, this will find the global max
            if A[i][mid] > max:
                # Save
                max = A[i][mid]
                new_i, new_mid = i, mid
    #less than +1
    if A[new_i][new_mid] < A[new_i][new_mid+1]:
        return two_peak(A, new_mid+1, hi)
    #equal to the hi
    elif new_mid == hi:
        return A[new_i][new_mid]
    # less than -1
    elif A[new_i][new_mid] < A[new_i][new_mid-1]:
        return two_peak(A, lo, new_mid-1)
    # Otherwise, this is the peak
    else:
        return A[new_i][new_mid]
        
print(two_peak(A, lo, hi))

17


The complexity of this function is O(nlogn) because it is O(rows * columns) and we iterated over n rows and logn columns.

**Question 5**

In [37]:
import time
import numpy as np

def selection_sort(A):
    for i in range(len(A)):
        min_idx = i
        for j in range(i + 1, len(A)):
            if A[min_idx] > A[j]:
                min_idx = j
        A[i], A[min_idx] = A[min_idx], A[i]
    return A

def insertion_sort(A):
    n = len(A)
    count = 0 
    for i in range(1,n):
        k = i
        while k > 0 and A[k] < A[k-1]:
            A[k], A[k-1] = A[k-1], A[k]
            k = k - 1
            count += 1
    return count, A

def merge(front, back):
    pos_f, pos_b = 0,0
    merged = np.zeros(len(front)+len(back))
    for i in range (len(merged)):
        if pos_f == len(front):
            merged[i] = back[pos_b]
            pos_b += 1
        elif pos_b == len(back):
            merged[i] = front[pos_f]
            pos_f += 1
        elif front[pos_f] < back[pos_b]:
            merged[i] = front[pos_f]
            pos_f += 1
        else:
            merged[i] = back[pos_b]
            pos_b += 1
    return merged

def merge_sort(A):
    n = len(A)
    if n <= 1:
        return A
    mid = int(n/2)
    front = merge_sort(A[0:mid])
    back = merge_sort(A[mid:])
    return merge(front, back)

def partition(A, lo, hi):
    pivotvalue = A[lo]
    i = lo+1    
    j = hi
    done = False
    while not done:
        while i <= j and A[i] <= pivotvalue:
            i = i + 1
        while A[j] >= pivotvalue and j >= i:
            j = j - 1
        if j < i:
            done = True
        else:
            A[i], A[j] = A[j], A[i]
    A[lo], A[j] = A[j], A[lo]
    return j, A

def quick_sort(A, lo, hi):
    if lo < hi:
        splitpoint, A = partition(A, lo, hi)
        A = quick_sort(A, lo, splitpoint - 1)
        A = quick_sort(A, splitpoint + 1, hi)    
    return A

def quick_sort_helper_rand(A):
    np.random.shuffle(A)
    return quick_sort(A, 0, len(A)-1)

def quick_sort_helper_nonrand(A):
    return quick_sort(A, 0, len(A)-1)

In [49]:
# Need to create a runtime function
import time
import numpy as np

def runtime(A, function):
    init = time.clock()
    function(A)
    fin = time.clock()
    time_in_sec = fin - init
    return time_in_sec

In [71]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

sorts = ['selection_sort', 'insertion_sort', 'merge_sort', 'quick_sort_helper_rand', 'quick_sort_helper_nonrand']

def doubling_exper(function):
    run_time = []
    size = []
    for n in sorts:
        perfectly_sorted = sorted(np.random.randint(0,100,size=2**n)) #A
        reversely_sorted = sorted(np.random.randint(0,100,size=2**n), reverse=True) #B
        random_order = np.random.randint(0,100,size=2**n) #C
        identical_ones = np.ones(2**n) #D
        distinct_random = np.random.randint(0,2,size=2**n) #E
        run = runtime(A, function)
        run_time.append(run)
        size.append(2**n)
    return run_time, size

sorts1 = [selection_sort, insertion_sort, merge_sort, quick_sort_helper_rand, quick_sort_helper_nonrand]
options1_mapped = {'perfectly_sorted': perfectly_sorted, 'reversely_sorted': reversely_sorted, 
                   'random_order': random_order, 'identical_ones': identical_ones, 'distinct_random': distinct_random}
for s in sorts1:
    for o in options1_mapped:
        run_time, size = doubling_exper(s)
        plt.plot(size, run_time, linewidth="1")
plt.show()

TypeError: unsupported operand type(s) for ** or pow(): 'int' and 'str'

**Question 6**

Part A

In [ ]:
'''
We shall use the strategy Kevin taught us in lab!

S(0) = 6, for n = 0
S(n) = S(n-1) + 2, for n = 1, 2, 3, ... 

Solving:
S(0) = 6 
S(1) = S(0) + 2 = 6 + 2 = 8
S(2) = S(1) + 2 = 8 + 2 = 10
S(3) = S(2) + 2 = 10 + 2 = 12
___
S(n) = S(n-1) + 2
S(n-1) = S(n-2) + 2
S(n-2) = S(n-3) + 2
S(n-3) = S(n-4) + 2

so then we put these equations in the original:
S(n) = (S(n-2) + 2) + 2
S(n) = (S(n-3) + 2) + 2 + 2
S(n) = (S(n-4) + 2) + 2 + 2 + 2
...
Eventually we arrive at S(n) = S(n-k) + 2*k
We put in the initial condition and we get:

THEREFORE: S(n) = 6 + 2n

'''


Part B

In [ ]:
'''
T(1) = 2, for n = 1
T(n) = 2*T(n-1) + 4, for n = 2, 3, 4, ...

Solving:
T(1) = 2
T(2) = 2*T(1) + 4 = 2*2 + 4 = 8
T(3) = 2*T(2) + 4 = 2*8 + 4 = 20
T(4) = 2*T(3) + 4 = 2*20 + 4 = 44
___
T(n) = 2*T(n-1) + 4
T(n-1) = 2*T(n-2) + 4
T(n-2) = 2*T(n-3) + 4
T(n-3) = 2*T(n-4) + 4

And putting into original:
T(n) = 2(2*T(n-2) + 4) + 4
T(n) = 2(2(2*T(n-3) + 4) + 4) + 4
T(n) = 2(2(2(2*T(n-4) + 4) + 4) + 4) + 4
...
Eventually, we arrive at T(n) = 2^k * T(n-k) + 4 * (2^k - 1)

Plug in initial conditions:
T(n) = 2^(n-1) * T(1) + 4 * (2^(n-1) - 1)
T(n) = 2^(n-1) * 2 + 4 * (2^(n-1) - 1)
T(n) = 2^(n) * 2 + 2^(n) - 4
T(n) = 2*2^(n) + 2^(n) - 4
T(n) = 2^(n)(2 + 1) - 4

THEREFORE: T(n) = 3*2^(n) - 4

'''

BONUS

In [ ]:
'''
Q(1) = c, for n = 1
Q(n) = Q(n/2) + 2n, for n = 2, 4, 8, ...

Solving:
Q(1) = c
___
Q(n) = Q(n/2) + 2n
Q(n/2) = Q(n/4) + n
Q(n/4) = Q(n/8) + n/2
Q(n/8) = Q(n/16) + n/4

So if we simplify further:
Q(n) = Q(n/4) + n + 2n
Q(n) = Q(n/8) + n/2 + n + 2n
Q(n) = Q(n/16) + n/4 + n/2 + n + 2n
...
Eventually we get: Q(n) = Q(n/2^k) + 4n(1-1/log(k))

THEREFORE: Q(n) = c + 4n(1 - 1/n)

'''

**Question 7**

In [ ]:
'''
T(1) = 0, for n = 1, (a subarray of 1 element requires no comparisons)
T(n) = 2T(n/2) + (n-1), for n = 2, 4, 8 ...
(derived from T(n/2) comparisons to get the first half of the array, another T(n/2) for the second half and n-1 to 
merge them)

Solving:
T(1) = 0
___
T(n) = 2*T(n/2) + (n-1)
T(n/2) = 2*T(n/4) + (n/2-1)
T(n/4) = 2*T(n/8) + (n/4-1)
T(n/8) = 2*T(n/16) + (n/8-1)

Now we start to simplify:
T(n) = 2*T(n/2) + (n-1)
T(n) = 2*(2*T(n/4) + (n/2-1)) + (n-1)
T(n) = 2*(2*(2*T(n/8) + (n/4-1)) + (n/2-1)) + (n-1)
T(n) = 2*(2*(2*(2*T(n/16) + (n/8-1)) + (n/4-1)) + (n/2-1)) + (n-1)

We see a pattern emerge:
T(n) = 2^k*T(n/2^k) + k*n - (2^k - 1)

Now we put in initial condition:
T(n) = 2^k*T(1) + k*n - (2^k - 1)
k = log_2(n)
so we have:
T(n) = n*T(1) + nlog_2(n) - (n - 1)
T(n) = n*0 + nlog_2(n) - n + 1

THEREFORE: T(n) = nlog_2(n) - n + 1

'''

**Question 8**

Part A

In [ ]:
'''
A = [13, 19, 9, 5, 9, 8, 11, 9, 6]
partition(A, lo = 0, hi = len(A)-1)

Swap 6 and 19
A = [13, 6, 9, 5, 9, 8, 11, 9, 19]

Swap 9 and 13
A = [9, 6, 9, 5, 9, 8, 11, 13, 19]

Can't really do anything else here. Not a great pivot...

'''

Part B

In [32]:
def quicksort_3way(A, lo, hi):
    if hi <= lo:
        return A
    lt, i, gt = lo, lo + 1, hi
    v = A[lo]
    while i <= gt:
        if A[i] < v:
            A[lt], A[i] = A[i], A[lt]
            i += 1
            lt += 1
        elif A[i] > v:
            A[i], A[gt] = A[gt], A[i]
            gt -= 1
        else:
            i += 1
    A = quicksort_3way(A, lo, lt - 1)
    A = quicksort_3way(A, gt + 1, hi)
    print('i =', i)
    return A

Part C

In [33]:
A = [2, 3, 1, 2, 2, 2, 3, 3]
quicksort_3way(A, 0, len(A) - 1)

i = 8
i = 5


[1, 2, 2, 2, 2, 3, 3, 3]

Here, you can see that i terminates on 5 because it will compare with 3, 1, 2, 2, 2, and then stop when it reaches that last '2' (since it will not swap anymore cause after that is 3, 3).

**Question 9**

In [34]:
def Funny_sort (A, i, j): 
    if A[i] > A[j]:
        A[i], A[j] = A[j], A[i]
    if (i + 1) >= j:
        return
    k = int((j-i+1)/3) # the "int" function acts as floor
    Funny_sort(A, i, j-k) 
    Funny_sort(A, i+k, j) 
    Funny_sort(A, i, j-k)
    return A

Part A

In [35]:
A = [2, 3, 1, 2, 2, 2, 3, 3]
Funny_sort(A, 0, len(A) - 1)

[1, 2, 2, 2, 2, 3, 3, 3]

Yes, this code correctly sorts the array because it sorts the first 2/3rds of the array and then the last 2/3rd of the array, then back to the first 2/3rds, which are already sorted. 

Part B

In [ ]:
'''
Since we are sorting in 2/3rds 3 times we have something like:
3 * 2n/3

The complexity is O(1)

THEREFORE: T(n) = 3*T(2n/3) + O(1)

'''

Part C

The runtime is worse than MergeSort